In [1]:
import requests
import openpyxl
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import csv
import pandas as pd

In [2]:
import requests
from bs4 import BeautifulSoup
data = []
text = 'https://www.bbc.co.uk'
for page_number in range(1, 37):
    url = f"https://www.bbc.co.uk/news/topics/cdl8n2edgj5t?page={page_number}"
    page = requests.get(url)
    if page.status_code == 200:
        soup = BeautifulSoup(page.content, "html.parser")
        articles = soup.find_all('a', class_='ssrcss-1mrs5ns-PromoLink exn3ah91')
        for article in articles:
            title_element = article['href']
            if title_element:
                title_text = text+title_element
                data.append(title_text)
#data

In [3]:
def convert_timestamp_format(timestamp):
    dt_object = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S.%fZ")
    formatted_date = dt_object.strftime("%m/%d/%y")
    return formatted_date


def extract_date_from_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        date_element = soup.find('time', {'data-testid': 'timestamp'})
        if date_element:
            timestamp = date_element['datetime']
            formatted_date = convert_timestamp_format(timestamp)
            return formatted_date
        
def scrape_articles(url):
    dat = []
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        articles = soup.find('div', class_='ssrcss-7uxr49-RichTextContainer e5tfeyi1')
        title_text = ""
        if articles:  
            article = articles.find_all('p')
            for ar in article:
                title_text += ar.text
    return title_text
        
def scrape_headlines_and_dates(page_urls):
    hd = []
    for url in page_urls:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            headline_element = soup.find('h1')
            date_text = extract_date_from_page(url)
            article = scrape_articles(url)
            if date_text:
                headline_text = headline_element.text.strip()
                hd.append([date_text,headline_text,article])
    return hd

result_data = scrape_headlines_and_dates(data)
#result_data

In [4]:
csv_file_path = 'BBC_summary_dataset_2024.csv'
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Date', 'Title', 'summary'])
    csv_writer.writerows(result_data)

In [5]:
dt = pd.read_csv('/kaggle/working/BBC_summary_dataset_2024.csv')
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 811 entries, 0 to 810
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Date     811 non-null    object
 1   Title    811 non-null    object
 2   summary  743 non-null    object
dtypes: object(3)
memory usage: 19.1+ KB
